In [ ]:
import os
import io
import glob
import json
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import pearsonr
import glob
import argparse


In [ ]:
from google.colab import drive
drive.mount('/content/drive')   

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/Favaloro/Tesis/ABIDE/time_series

/content/drive/.shortcut-targets-by-id/1lco1rMbsSCWuDnBkEXF4SL0ppvvDGczt/ABIDE/time_series


In [ ]:
path_data = "/content/drive/MyDrive/Favaloro/Tesis/ABIDE/time_series"
path_ABIDE = "/content/drive/MyDrive/Favaloro/Tesis/ABIDE"


In [ ]:
# coordenadas ROIs
#coords = pd.read_csv(os.path.join(path_ABIDE, "rois_coordinates.csv"))

# labels ROIs
#labels = np.loadtxt(os.path.join(path_ABIDE, "rois_labels.txt"), dtype="str")

# metadata AUTISTA/CONTROL
metadata = pd.read_csv(os.path.join(path_ABIDE, "metadata.csv"))


In [ ]:
# Tabla de frecuencias
frequencies =  metadata["condition"].value_counts()

table = pd.DataFrame({
    "condition": frequencies.index,
    "Frecuencia": frequencies.values
})
print(table)

In [ ]:
vectores = []
matrices = []
condicion = []
ids = []

# itera por cada directorio (cada sujeto) dentro de la carpeta time_series
for carpeta_suj in os.listdir(path_data):
    if not os.path.isdir(os.path.join(path_data, carpeta_suj)):
        continue

    # subject_id esta en el nombre de la carpeta
    subject_id = carpeta_suj
    #print(subject_id)



    # Verificar si el subject_id corresponde a la condición "CONT" o "AUT"
    if subject_id in metadata[metadata['condition'].isin(['CONT', 'AUT'])]['subject_id'].values:

      ids.append(subject_id)
      #condicion.append(metadata[metadata['subject_id']==subject_id].condition)

      condicion.append(metadata[metadata['subject_id']==subject_id]['condition'].values[0])

      #print(metadata[metadata['subject_id']==subject_id].condition.values[0])




      # leer el csv con la serie temporal de cada sujeto
      sujeto_ts = pd.read_csv(os.path.join(path_data, carpeta_suj, "tseries.csv"), header=None)

      #print(sujeto_ts.shape)
      # matriz de correlacion de Pearson
      corr_matriz = np.corrcoef(sujeto_ts)
      #print(corr_matriz.shape)
      matrices.append(corr_matriz)

      # selecciono el triangulo inferior de la matriz de correlaciones de conectividad
      #triangulo_inf = np.tril(corr_matriz, k=-1)
      #triangulo_inf = np.tril_indices(corr_matriz, k=-1)


      # Vectorizar el triángulo inferior
      #vector_inf = np.ravel(triangulo_inf)
      vector_inf = corr_matriz[np.triu_indices_from(corr_matriz, k=1)]    #triu selecciona el sup k ignora la diagoanl
      #print(vector_inf.shape)
      #print(vector_inf)

    # Agregar el vector a la lista de vectores
      
      vectores.append(vector_inf)
      #print(vectores)


In [ ]:
# crear df de la lista de vectores
df_vectores = pd.DataFrame(vectores)
# agregar la condicion al df vectores
df_vectores['condicion'] = condicion

#df_vectores['subject_id'] = ids

#df_vectores = pd.DataFrame({'subject_id': ids, 'condition': condicion, 'vector': vectores})

# Guardar el DataFrame en un archivo CSV

In [ ]:
df_vectores

,0,1,2,3,4,5,6,7,8,9,...,4362,4363,4364,4365,4366,4367,4368,4369,4370,condicion
0,0.884496,0.709854,0.759467,0.808396,0.748825,0.746855,0.797611,0.754630,0.699556,0.558432,...,0.894507,0.848783,0.875080,0.853000,0.754495,0.673811,0.807199,0.812130,0.915447,AUT
1,0.913155,0.508644,0.210797,0.375724,0.136328,0.550674,0.217562,0.446146,0.257435,0.336017,...,0.552078,0.645986,0.629501,0.814782,0.648253,0.585712,0.574145,0.465562,0.773946,CONT
2,0.693080,0.639012,0.427046,0.553249,0.370843,0.639087,0.615761,0.474038,0.523910,0.360967,...,0.697781,0.573982,0.727019,0.648158,0.601135,0.471724,0.460433,0.525508,0.854126,AUT
3,0.674283,0.356046,0.395826,0.250278,0.315492,0.327619,0.418142,0.318003,0.344895,0.407478,...,0.762027,0.589437,0.729525,0.827515,0.567599,0.546411,0.680090,0.719204,0.798524,CONT
4,0.631215,0.468764,0.447137,0.464821,0.383575,0.549706,0.380909,0.574176,0.417614,0.567322,...,0.634173,0.671634,0.658479,0.599554,0.443173,0.456866,0.421308,0.572873,0.629786,CONT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
968,0.888090,0.765874,0.792513,0.796749,0.828143,0.749775,0.779592,0.735686,0.744654,0.599306,...,0.654844,0.773923,0.797517,0.470793,0.468013,0.448449,0.602436,0.582362,0.860097,AUT
969,0.863935,0.567457,0.385698,0.421088,0.392381,0.485603,0.474980,0.413237,0.405421,0.474576,...,0.449595,0.649403,0.753157,0.676551,0.541748,0.504960,0.538907,0.444945,0.843787,CONT
970,0.739739,0.603254,0.720546,0.589603,0.650524,0.595682,0.587275,0.539085,0.547962,0.529656,...,0.778980,0.643013,0.668551,0.580668,0.419678,0.308983,0.424211,0.498261,0.808581,AUT
971,0.565503,0.291143,0.493934,0.175546,0.331135,0.258164,0.490494,0.079805,0.200938,0.333647,...,0.583292,0.622685,0.685849,0.413688,0.462261,0.307173,0.357695,0.390904,0.658923,AUT


In [ ]:
df_vectores.to_csv('vectores.csv', index=False)


# Carga DF vectores

In [ ]:
# Importar el csv desde disco local
from google.colab import files
uploaded = files.upload()

Saving vectores.csv to vectores (1).csv


In [ ]:
import io
df = pd.read_csv(io.BytesIO(uploaded['vectores.csv']))
# conertir el ds en un pandas df
print(df)

            0         1         2         3         4         5         6  \
0    0.884496  0.709854  0.759467  0.808396  0.748825  0.746855  0.797611   
1    0.913155  0.508644  0.210797  0.375724  0.136328  0.550674  0.217562   
2    0.693080  0.639012  0.427046  0.553249  0.370843  0.639087  0.615761   
3    0.674283  0.356046  0.395826  0.250278  0.315492  0.327619  0.418142   
4    0.631215  0.468764  0.447137  0.464821  0.383575  0.549706  0.380909   
..        ...       ...       ...       ...       ...       ...       ...   
968  0.888090  0.765874  0.792513  0.796749  0.828143  0.749775  0.779592   
969  0.863935  0.567457  0.385698  0.421088  0.392381  0.485603  0.474980   
970  0.739739  0.603254  0.720546  0.589603  0.650524  0.595682  0.587275   
971  0.565503  0.291143  0.493934  0.175546  0.331135  0.258164  0.490494   
972  0.794437  0.456195  0.612261  0.567565  0.430790  0.597022  0.561141   

            7         8         9  ...      4362      4363      4364  \
0  

Como puede observarse, vemos que hay 4371 columnas que corresponden a los vectores generados, que contienen  todas las regiones (parcelas). 
Cada fila corresponde a un sujeto, que cuenta con su condicion correspondiente en la columna "condicion"

In [ ]:
### Random forest

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

features = df.drop(['condicion'], axis=1)

labels = df['condicion'].values  

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)

# Crear una instancia del clasificador Random Forest
rf_classifier = RandomForestClassifier()

# Entrenar el modelo utilizando los datos de entrenamiento
rf_classifier.fit(X_train, y_train)

# Predecir las etiquetas para los datos de prueba
y_pred = rf_classifier.predict(X_test)

# Calcular la precisión de la clasificación
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.7025641025641025


In [ ]:
# optimizacion de hiperparametros

In [ ]:
from sklearn.metrics import confusion_matrix

confusion_mat = confusion_matrix(y_test, y_pred)
print(confusion_mat)


[[ 33  47]
 [ 11 104]]


In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score

precision = precision_score(y_test, y_pred, average=None)
recall = recall_score(y_test, y_pred, average=None)
f1 = f1_score(y_test, y_pred, average=None)

print("Precisión por condición:", precision)
print("Recall por clase:", recall)
print("F1-score por clase:", f1)


Precisión por clase: [0.75       0.68874172]
Recall por clase: [0.4125     0.90434783]
F1-score por clase: [0.53225806 0.78195489]


In [ ]:
# en todos los casos seria primero AUT y despues CONT xq es directo por orden alfabetico o como podria saberlo?
# poner interpretaion de los datos anteriores

In [ ]:
# lista matrices de conectividad a  numpy array
matrices = np.array(matrices)

In [ ]:
import tensorflow as tf
from tensorflow import keras

# Definir la arquitectura de la red convolucional
model = keras.Sequential([
    keras.layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu', input_shape=(N, N, 1)),  # COMO ESTABLECER ACA E SHAPE???
    keras.layers.MaxPooling2D(pool_size=(2, 2)),
    keras.layers.Flatten(),
    keras.layers.Dense(units=64, activation='relu'),
    keras.layers.Dense(units=2, activation='softmax')
])

# Compilar el modelo
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Entrenar el modelo
model.fit(X_train, y_train, epochs=10)    # deberia agregar la validacion?????

# Evaluar el modelo en el conjunto de prueba
loss, accuracy = model.evaluate(X_test, y_test)
print('Test Loss:', loss)
print('Test Accuracy:', accuracy)


Epoch 1/10


ValueError: ignored